In [1]:
from astropy import units as u
from astropy import constants as const
import numpy as np
import matplotlib.pyplot as plt
from scipy import integrate
from scipy import special
from scipy.optimize import curve_fit
from astropy.modeling.models import BlackBody
import pandas as pd


In [2]:
def load_partition_data():
    global partition_species
    global partition_coef
    global partition_data
    
    df    = pd.read_csv('RepairedPartitionFunctions.txt',header=None,sep=" ")
    df    = df.replace('-',np.nan)
    tempy = df.to_numpy()
    b     = tempy[:,1:-1]
    s     = tempy[:,0]

    partition_data    = b.astype(float)
    partition_species = s.astype(str)

    ## interpolating
    theta_columns       = np.linspace(.2,2.0,num=10)  # to match with the input data
    num_rows            = np.shape(b)[0]
    partition_coef      = np.zeros([num_rows,3])
    
    def test_func(x, a, b, c):
        return a * np.exp(-b * x) + c
    
    for i in range(num_rows):
        idx               = np.isfinite(theta_columns) & np.isfinite(partition_data[i])
        #partition_coef[i] = np.polyfit(theta_columns[idx],partition_data[i][idx],7)
        try:
            partition_coef[i], param_cov = curve_fit(test_func, theta_columns[idx], partition_data[i][idx])
        except:
            pass
        

In [3]:
def partition(temp,specy):
    # load partition info
    if specy == 'H-' or specy == 'HII' or specy == 'Li+':
        if isinstance(temp, int) or isinstance(temp, float):
            return 1
        else:
            return np.ones(temp.size)
    else:
        # find the species
        spec_index = np.where(partition_species == specy)[0][0]
    
        # interpolate function
        #spec_function = np.poly1d(partition_coef[spec_index])
        theta         = temptotheta(temp)
        param = partition_coef[spec_index]
        ans = param[0] * np.exp(-param[1] * theta) + param[2]
        return np.power(10,ans)
        #return ans

In [4]:
def Phi(temp,spec):
    # temperature dependent part of RHS Saha
    # find next ionization
    if spec == 'H-':
        nextspec = 'H'
    elif spec == 'H':
        nextspec = 'HII'
    else:
        nextspec = spec + '+'
    u0    = partition(temp,spec)
    u1    = partition(temp,nextspec)
    theta = temptotheta(temp)
    I     = X(spec)
    return (1.2024*10**9)*(u1/u0)*np.power(theta,-2.5)*np.power(10,-theta*I)

In [5]:
def X(specy):
    if specy == 'H-':
        return 0.755
    else:
        # check nist
        nist_index = np.where(nist_data == specy)[0][0]
        if nist_index>0:
            #use nist
            return nist_data[nist_index,2]
        else:
            #use grey        
            #how many +s
            numplus= len(specy)-specy.find('+')
            if specy.find('+')>0:
                specy = specy[:-numplus]
            else:
                numplus=0
    
            spec_index = np.where(ionization_data == specy)[0][0]
            
            return ionization_data[spec_index,numplus+2]

In [6]:
def load_ionization_data():
    global ionization_data
    global nist_data
    
    df2             = pd.read_fwf('ioniz.txt',header=None)  
    iontemp         = df2.to_numpy()
    ionization_data = iontemp[:,1:]

    df3 = pd.read_csv('nist_ioniz2.csv',header=None,sep = '\t')
    nisttemp = df3.to_numpy()
    nist_data = nisttemp[:,1:]
    testing = nist_data[:,0]
    nist_data[:,0] = np.char.strip(testing.astype(str))
    

In [7]:
def thetatotemp(theta):
    return 5040/theta

In [8]:
def temptotheta(temp):
    return 5040/temp

In [9]:
load_partition_data()
load_ionization_data()

/home/crobles/anaconda3/lib/python3.11/site-packages/scipy/optimize/_minpack_py.py:1010: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


IndexError: index 0 is out of bounds for axis 1 with size 0

In [ ]:
#temperatures = np.array([1000,2000,3000,4000,5000,6000,7000])

spec_list = ['H','Li','Ar','Fe']

for species in spec_list:
    
    test_theta = np.linspace(.2,2,num=1000)
    test_theta2 = np.linspace(.2,2,num=10)
    test_temp = thetatotemp(test_theta)
    species_index=np.where(partition_species == species)[0][0]
    u = partition(test_temp,species)
    plt.scatter(test_theta2,np.power(10,partition_data[species_index]))
    #plt.scatter(test_theta2,partition_data[species_index])
    plt.plot(test_theta,u)
    plt.title("Partition function for "+species)
    plt.xlabel(r"$\theta$")
    plt.ylabel(r"u($\theta$)")
    plt.legend(["partition data","exponential fit"])
    plt.savefig('hw6_partition_'+species+'.svg', bbox_inches='tight')
    #plt.yscale("log")
    plt.show()

In [ ]:
temp=np.linspace(2000,10000,num=100)

frac1 = Phi(temp,'Fe')
frac2 = Phi(temp,'Fe+')
ones = np.ones(temp.size)

zero = (ones/frac1)/((ones/frac1)+ones+frac2)
first = ones/((ones/frac1)+ones+frac2)
second = frac2/((ones/frac1)+ones+frac2)


plt.plot(temp,zero)
plt.plot(temp,first)
plt.plot(temp,second)
plt.title("Ionization of Iron")
plt.xlabel("Temperature [K]")
plt.ylabel("Fraction in Ionization Stage")
plt.legend(["Neutral","1st Ion","2nd Ion"])
plt.show()

In [ ]:
print("hi"+"hi")